In [1]:
import csv
import io
import os
import pandas as pd
import logging
import numpy as np

import apache_beam as beam
import apache_beam.runners.interactive.interactive_beam as ib
from apache_beam.runners.interactive.interactive_runner import InteractiveRunner
from apache_beam.options import pipeline_options
from apache_beam.options.pipeline_options import GoogleCloudOptions
import google.auth

In [2]:
project_id = 'teste-dotz-292803:dotz'

In [3]:
def get_json_data(url):
    import json
    with open(url) as json_file:
        data = json.load(json_file)

    return data

In [12]:
sch = None
def create_dataframe(readable_file):
    # Open a channel to read the file from GCS
    gcs_file = beam.io.filesystems.FileSystems.open(readable_file)

    # Read it as csv, you can also use csv.reader
    csv_dict = csv.DictReader(io.TextIOWrapper(gcs_file))

    # Create the DataFrame
    df = pd.DataFrame(csv_dict)
    df = df[['component_id','component_type_id']].replace({np.nan:-1})
    #print(df.to_string())
    
    #df.head()
    sch = pd.io.json.build_table_schema(df)
    
    return df.to_dict('records')

In [13]:
df = pd.read_csv(f"{ os.getcwd() }/dataset/comp_boss.csv", sep=',')
sch = pd.io.json.build_table_schema(df)
print(sch['fields'])

[{'name': 'index', 'type': 'integer'}, {'name': 'component_id', 'type': 'string'}, {'name': 'component_type_id', 'type': 'string'}, {'name': 'type', 'type': 'string'}, {'name': 'connection_type_id', 'type': 'string'}, {'name': 'outside_shape', 'type': 'string'}, {'name': 'base_type', 'type': 'string'}, {'name': 'height_over_tube', 'type': 'number'}, {'name': 'bolt_pattern_long', 'type': 'number'}, {'name': 'bolt_pattern_wide', 'type': 'number'}, {'name': 'groove', 'type': 'string'}, {'name': 'base_diameter', 'type': 'number'}, {'name': 'shoulder_diameter', 'type': 'number'}, {'name': 'unique_feature', 'type': 'string'}, {'name': 'orientation', 'type': 'string'}, {'name': 'weight', 'type': 'number'}]


In [148]:
from apache_beam.io.gcp.internal.clients import bigquery
table_spec = bigquery.TableReference(
    projectId='teste-dotz-292803',
    datasetId='dotz',
    tableId='xpto')

In [ ]:
additional_bq_parameters = {
  'timePartitioning': {'type': 'DAY'},
  'clustering': {'fields': ['country']}}

with beam.Pipeline() as p:
    elements = (p | beam.Create([
    {'country': 'mexico', 'query': 'acapulco'},
    {'country': 'canada', 'query': 'influenza'},
    ]))

    elements | beam.io.gcp.bigquery.WriteToBigQuery(
    table='teste-dotz-292803:dotz.yyyy',
    #additional_bq_parameters=additional_bq_parameters,
        schema='country:STRING, query:STRING',
     method="STREAMING_INSERTS")

In [14]:
p = beam.Pipeline()
boss = (  p | beam.Create(['dataset/comp_boss.csv'])
            | beam.FlatMap(create_dataframe)
            #| beam.io.WriteToBigQuery(table=lambda row, table_dict: table_dict[row['type']] ,
            #                          table_side_inputs='teste-dotz-292803:dotz.comp_boss')
        
            #| beam.io.WriteToBigQuery(table_spec,
            #                          schema=sch,
            #                          write_disposition=beam.io.BigQueryDisposition.WRITE_TRUNCATE,
            #                          create_disposition=beam.io.BigQueryDisposition.CREATE_IF_NEEDED)
        
        
            | beam.io.gcp.bigquery.WriteToBigQuery(
                        table='teste-dotz-292803:dotz.component',
                        schema='component_id:STRING, component_type_id:STRING',
                        method="STREAMING_INSERTS")
        
            #| 'Write' >> beam.io.WriteToText('out/comp')
            
            #| 'save' >> beam.io.Write(
            #   beam.io.BigQuerySink(
            #   'teste-dotz-292803:dotz.comp_boss',
            #   schema=sch,
            #   create_disposition=beam.io.BigQueryDisposition.CREATE_IF_NEEDED,
            #   write_disposition=beam.io.BigQueryDisposition.WRITE_TRUNCATE))

)
p.run()
print(boss)

{'FailedRows': <PCollection[[14]: WriteToBigQuery/_StreamToBigQuery/StreamInsertRows/ParDo(BigQueryWriteFn).FailedRows] at 0x20d83509448>}


In [87]:
p = beam.Pipeline(runner=InteractiveRunner())
boss = p | 'Read comp_boss' >> beam.io.ReadFromText('dataset/comp_boss.csv', skip_header_lines=1)
bom = p | 'Read Bill_of_Material' >> beam.io.ReadFromText('dataset/bill_of_materials.csv', skip_header_lines=1)
price = p | 'Read Price_Quote' >> beam.io.ReadFromText('dataset/price_quote.csv', skip_header_lines=1)
pop = p | 'pop' >> beam.Create(get_json_data('dataset/populacao.json'))

In [93]:
ib.options.capture_duration = timedelta(seconds=60)
ib.show(pop)


NameError: name 'timedelta' is not defined

In [29]:
inputs_pattern = 'dataset/comp_boss.csv'
outputs_prefix = 'outputs/part'

boss= None
price = None

with beam.Pipeline() as pipeline:
    boss = (
        pipeline
        | 'Read File' >> beam.io.ReadFromText(inputs_pattern)
        #| 'Write' >> beam.io.WriteToText(outputs_prefix)
        | 'teste' >> beam.Map(boss)
    )

In [14]:
print(boss)

PCollection[[12]: Read File/Read/_SDFBoundedSourceWrapper/ParDo(SDFBoundedSourceDoFn).None]


In [30]:
p = beam.Pipeline()

In [136]:
additional_bq_parameters = {
  'timePartitioning': {'type': 'DAY'},
  'clustering': {'fields': ['country']}}

with beam.Pipeline() as p:
    elements = (p | beam.Create([
    {'country': 'mexico', 'query': 'acapulco'},
    {'country': 'canada', 'query': 'influenza'},
    ]))

    elements | beam.io.gcp.bigquery.WriteToBigQuery(
    table='teste-dotz-292803:dotz.yyyy',
    #additional_bq_parameters=additional_bq_parameters,
        schema='country:STRING, query:STRING',
     method="STREAMING_INSERTS")

ERROR:apache_beam.runners.direct.executor:Exception at bundle <apache_beam.runners.direct.bundle_factory._Bundle object at 0x000002A16193CBC8>, due to an exception.
 Traceback (most recent call last):
  File "e:\projetos\jobs\dataengineer\dotz\env\lib\site-packages\apache_beam\runners\direct\executor.py", line 382, in call
    finish_state)
  File "e:\projetos\jobs\dataengineer\dotz\env\lib\site-packages\apache_beam\runners\direct\executor.py", line 423, in attempt_call
    result = evaluator.finish_bundle()
  File "e:\projetos\jobs\dataengineer\dotz\env\lib\site-packages\apache_beam\runners\direct\transform_evaluator.py", line 1184, in finish_bundle
    writer.Write(v.value)
  File "e:\projetos\jobs\dataengineer\dotz\env\lib\site-packages\apache_beam\io\gcp\bigquery_tools.py", line 1218, in __exit__
    self._flush_rows_buffer()
  File "e:\projetos\jobs\dataengineer\dotz\env\lib\site-packages\apache_beam\io\gcp\bigquery_tools.py", line 1204, in _flush_rows_buffer
    (self.project_id,